In [97]:
from typing import List, Dict, Union

import simpy
import pandas as pd
import pybamm

pybamm.set_logging_level("INFO")

In [98]:
class PybammBattery:

    def __init__(self, env, capacity, soc=0):
        """
        'Nominal cell capacity [A.h]' is set twice the capacity provided, so that we can easily calculate the excess power.
        Otherwise the excess energy cannot be calculated since the battery won't charge pass the capacity.
        """
        self.env = env
        self.capacity = capacity
        self.soc = soc
        self.excess_power = 0
        
        self.step_solution = None
        self.step_solver = pybamm.CasadiSolver()
        
        # load model
        self.model = pybamm.lithium_ion.SPMe()
        
        # create geometry
        geometry = self.model.default_geometry
        
        # load parameter values and process model and geometry
        param = self.model.default_parameter_values
        param['Current function [A]'] = '[input]'
        param.process_model(self.model)
        param.process_geometry(geometry)
        
        # cerate mesh
        mesh = pybamm.Mesh(geometry, self.model.default_submesh_types, self.model.default_var_pts)
        
        # discretise model
        disc = pybamm.Discretisation(mesh, self.model.default_spatial_methods)
        disc.process_model(self.model)
        
        # save input_parameters
        Vmin = param['Lower voltage cut-off [V]']
        Vmax = param['Upper voltage cut-off [V]']
        Cn = param.evaluate(self.model.param.C_n_init)
        Cp = param.evaluate(self.model.param.C_p_init)
        self.input_parameters = {"V_min": Vmin, "V_max": Vmax, "C_n": Cn, "C_p": Cp}

    def update(self, current):
        if current == 0:
            return 0;
        
        dt = 60
        self.input_parameters['Current function [A]'] = current
        self.step_solution = self.step_solver.step(self.step_solution, self.model, dt=dt, npts=2, inputs=self.input_parameters)

In [99]:
def simulate(env: simpy.Environment, battery: PybammBattery, power_delta_list: List[float], records: List[Dict]):
    
    for power_delta in power_delta_list:
        yield env.timeout(1)
        battery.update(power_delta)
        records.append({
            "power_delta": power_delta,
            "excess_power": battery.excess_power,
            "soc": battery.soc,
            "capacity [A.h]": battery.capacity
        })

In [100]:
# For now let's assume the simple case of one step every second where we first (dis)charge and then implicitly read.
# Later we can extend this to a more asynchronous charge/discharge/read pattern with different processes if we want
power_delta_list = [5, -3, 2, 3, 4, -2]
records = []  # log of some infos for later analysis

env = simpy.Environment()
battery = PybammBattery(env, capacity=5)
env.process(simulate(env, battery, power_delta_list, records))
env.run()

result = pd.DataFrame(records)
with open("result.csv", "w") as f:
    f.write(result.to_csv())
print(result)

2022-04-06 16:16:08,359 - [INFO] base_battery_model.build_model(834): Start building Single Particle Model with electrolyte
2022-04-06 16:16:08,443 - [INFO] base_battery_model.build_model(854): Finish building Single Particle Model with electrolyte
2022-04-06 16:16:08,471 - [INFO] parameter_values.process_model(415): Start setting parameters for Single Particle Model with electrolyte
2022-04-06 16:16:08,587 - [INFO] parameter_values.process_model(518): Finish setting parameters for Single Particle Model with electrolyte
2022-04-06 16:16:08,589 - [INFO] discretisation.process_model(137): Start discretising Single Particle Model with electrolyte
2022-04-06 16:16:08,849 - [INFO] discretisation.process_model(254): Finish discretising Single Particle Model with electrolyte
2022-04-06 16:16:08,851 - [INFO] base_solver.set_up(111): Start solver set-up
2022-04-06 16:16:08,890 - [INFO] base_solver.set_up(678): Finish solver set-up


   power_delta  excess_power  soc  capacity [A.h]
0            5             0    0               5
1           -3             0    0               5
2            2             0    0               5
3            3             0    0               5
4            4             0    0               5
5           -2             0    0               5


In [94]:
battery.model.default_parameter_values.evaluate(battery.model.param.C_p)

0.0442505104809096

In [95]:
battery.model.default_parameter_values['Lower voltage cut-off [V]']

3.105

In [96]:
battery.model.default_parameter_values['Upper voltage cut-off [V]']

4.1

In [102]:
battery.step_solution.summary_variables

In [103]:
battery.step_solution.data

{}